# Dependencies
---


In [1]:
from pathlib import Path
import sqlite3

import pandas as pd
import requests


# Extract
**Download original CSVs**
<br><br>
_The English Women's Football (EWF) Database, https://github.com/probjects/ewf-database._

---


In [12]:
github = r'https://github.com/probjects/ewf-database/raw/refs/heads/main/data/'
local = Path(r'data/')

filenames = ['ewf_appearances.csv', 'ewf_matches.csv', 'ewf_standings.csv']

for filename in filenames:
    filepath = local / filename

    if not filepath.exists():
        url = github + filename
        response = requests.get(url)
        
        if response.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(response.content)
        else:
            print(f"Download failed for {filename!r}: {response.status_code} {response.reason}")

    else:
        print(f"Filename {filename!r} already exists")


Filename 'ewf_appearances.csv' already exists
Filename 'ewf_matches.csv' already exists
Filename 'ewf_standings.csv' already exists


# Transform
**Convert to a normalized relational schema**

---


## Original Schema

In [3]:
df_a = pd.read_csv('data/ewf_appearances.csv')
df_m = pd.read_csv('data/ewf_matches.csv')
df_s = pd.read_csv('data/ewf_standings.csv')


In [4]:
df_a.head()

,season_id,season,tier,division,match_id,match_name,date,attendance,team_id,team_name,...,away_team,goals_for,goals_against,goal_difference,result,win,loss,draw,note,points
0,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,2011-04-13,"2,510",T-008-T,Chelsea Ladies,...,0,0,1,-1,Loss,0,1,0,NaN,0
1,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,2011-04-13,"2,510",T-001-T,Arsenal Ladies,...,1,1,0,1,Win,1,0,0,NaN,3
2,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,2011-04-13,742,T-016-T,Lincoln Ladies,...,0,0,1,-1,Loss,0,1,0,NaN,0
3,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,2011-04-13,742,T-011-T,Doncaster Rovers Belles,...,1,1,0,1,Win,1,0,0,NaN,3
4,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-003-M,Birmingham City Ladies vs Bristol Academy,2011-04-14,602,T-003-T,Birmingham City Ladies,...,0,4,0,4,Win,1,0,0,NaN,3


In [5]:
df_m.head()

,season_id,season,tier,division,match_id,match_name,date,attendance,home_team_id,home_team_name,...,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,home_team_win,away_team_win,draw,result,note
0,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,2011-04-13,"2,510",T-008-T,Chelsea Ladies,...,0 -- 1,0,1,-1,1,0,1,0,Away team win,NaN
1,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,2011-04-13,742,T-016-T,Lincoln Ladies,...,0 -- 1,0,1,-1,1,0,1,0,Away team win,NaN
2,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-003-M,Birmingham City Ladies vs Bristol Academy,2011-04-14,602,T-003-T,Birmingham City Ladies,...,4 -- 0,4,0,4,-4,1,0,0,Home team win,NaN
3,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-004-M,Liverpool Ladies vs Everton Ladies,2011-04-14,835,T-017-T,Liverpool Ladies,...,3 -- 3,3,3,0,0,0,0,1,Draw,NaN
4,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-005-M,Everton Ladies vs Doncaster Rovers Belles,2011-04-20,220,T-013-T,Everton Ladies,...,1 -- 1,1,1,0,0,0,0,1,Draw,NaN


In [6]:
df_s.head()

,season_id,season,tier,division,position,team_id,team_name,played,wins,draws,losses,goals_for,goals_against,goal_difference,points,point_adjustment,season_outcome
0,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),1,T-001-T,Arsenal Ladies,14,10,2,2,29,9,20,32,0,No change
1,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),2,T-003-T,Birmingham City Ladies,14,8,5,1,29,13,16,29,0,No change
2,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),3,T-013-T,Everton Ladies,14,7,4,3,19,13,6,25,0,No change
3,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),4,T-016-T,Lincoln Ladies,14,6,3,5,18,16,2,21,0,No change
4,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),5,T-006-T,Bristol Academy,14,4,4,6,14,20,-6,16,0,No change


## Normalize
**Split the data into 6+ tables:**
- seasons
- tiers
- teams (+name history)
- standings
- matches
- appearances

**Relationships:**
- [1:N] `tiers.tier_id -< seasons.season_id`


In [ ]:
# In the data, a "Tier" consists of `tier_id` (currently written as `tier`) and `division`
# Since the division name changes over time, use the most recent one to reflect the current state
# Choosing not to preserve the historic division names for this project

# Make a copy of original
df_tiers = df_s.copy()

# Rename column
tier_name_map = {'tier': 'tier_id'}
df_tiers.rename(columns=tier_name_map, inplace=True)

# Sort chronologically
df_tiers.sort_values(by=['tier_id', 'season_id'], ascending=[True, True], inplace=True)

# Filter
df_tiers.drop_duplicates(subset='tier_id', keep='last', inplace=True)
df_tiers = df_tiers[['tier_id', 'division']]

df_tiers


,tier_id,division
269,1,Women's Super League (WSL)
280,2,Women's Championship


In [ ]:
# Since both tiers share the same season (albeit different starting dates), it can be reduced further
# Easiest option is to truncate the existing `season_id`: `S-2021-2022-2-S` -> `S-2021-2022`
# This makes it easier to switch between divisions within the same season

df_seasons = df_s.copy()
df_seasons['season_id'] = df_seasons['season_id'].str.slice(start=0, stop=-4)

# Extract the start/end year
df_seasons[['year_start', 'year_end']] = df_seasons['season'].str.split('-', expand=True).astype(int)
df_seasons = df_seasons[['season_id', 'year_start', 'year_end']].drop_duplicates(subset='season_id', keep='last')

df_seasons


,season_id,year_start,year_end
7,S-2011-2011,2011,2011
15,S-2012-2012,2012,2012
23,S-2013-2013,2013,2013
41,S-2014-2014,2014,2014
59,S-2015-2015,2015,2015
78,S-2016-2016,2016,2016
97,S-2017-2017,2017,2017
117,S-2017-2018,2017,2018
139,S-2018-2019,2018,2019
162,S-2019-2020,2019,2020


In [83]:
# A "Team" consists of `team_id` and (current) `team_name`
# Some team names have changed over time so similar steps as above to get table reflecting current state

df_teams = df_s.sort_values(by=['team_id', 'season_id'])[['team_id', 'team_name']].drop_duplicates(subset='team_id', keep='last')
df_teams


,team_id,team_name
259,T-001-T,Arsenal Women
263,T-002-T,Aston Villa Women
271,T-003-T,Birmingham City Women
279,T-004-T,Blackburn Rovers Women
262,T-005-T,Brighton and Hove Albion Women
275,T-006-T,Bristol City Women
272,T-007-T,Charlton Athletic Women
258,T-008-T,Chelsea Women
233,T-009-T,Coventry United Women
269,T-010-T,Crystal Palace Women


In [92]:
# Store historical team names
df_team_history = df_s.copy()
df_team_history['season_id'] = df_team_history['season_id'].str.slice(start=0, stop=-4)

# Get current season
current_season = df_team_history['season_id'].sort_values().iloc[-1]

# Helpful to know when the team name was valid
df_team_history['season_start'] = df_team_history['season_id']
df_team_history['season_end'] = df_team_history['season_id']

team_name_start = df_team_history.groupby(['team_id', 'team_name'])['season_start'].min()
team_name_end = df_team_history.groupby(['team_id', 'team_name'])['season_end'].max()

df_team_history = team_name_start.to_frame().join(team_name_end).reset_index()

# If team name is still active, `season_end` should be blank
df_team_history['season_end'] = df_team_history['season_end'].mask(
    df_team_history['season_end']==current_season,
    pd.NA
)

df_team_history


,team_id,team_name,season_start,season_end
0,T-001-T,Arsenal Ladies,S-2011-2011,S-2017-2017
1,T-001-T,Arsenal Women,S-2017-2018,<NA>
2,T-002-T,Aston Villa Ladies,S-2014-2014,S-2018-2019
3,T-002-T,Aston Villa Women,S-2019-2020,<NA>
4,T-003-T,Birmingham City Ladies,S-2011-2011,S-2017-2018
5,T-003-T,Birmingham City Women,S-2018-2019,<NA>
6,T-004-T,Blackburn Rovers Ladies,S-2019-2020,S-2022-2023
7,T-004-T,Blackburn Rovers Women,S-2023-2024,<NA>
8,T-005-T,Brighton and Hove Albion Women,S-2017-2017,<NA>
9,T-006-T,Bristol Academy,S-2011-2011,S-2015-2015


In [95]:
# With information already capture in the above tables, can remove redundant information
df_standings = df_s.rename(columns=tier_name_map)
df_standings['season_id'] = df_standings['season_id'].str.slice(start=0, stop=-4)

df_standings.drop(columns=['season', 'division', 'team_name'], inplace=True)

df_standings


,season_id,tier_id,position,team_id,played,wins,draws,losses,goals_for,goals_against,goal_difference,points,point_adjustment,season_outcome
0,S-2011-2011,1,1,T-001-T,14,10,2,2,29,9,20,32,0,No change
1,S-2011-2011,1,2,T-003-T,14,8,5,1,29,13,16,29,0,No change
2,S-2011-2011,1,3,T-013-T,14,7,4,3,19,13,6,25,0,No change
3,S-2011-2011,1,4,T-016-T,14,6,3,5,18,16,2,21,0,No change
4,S-2011-2011,1,5,T-006-T,14,4,4,6,14,20,-6,16,0,No change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,S-2024-2025,2,7,T-027-T,20,9,3,8,30,34,-4,30,0,No change
277,S-2024-2025,2,8,T-026-T,20,5,6,9,22,25,-3,21,0,No change
278,S-2024-2025,2,9,T-033-T,20,3,4,13,16,48,-32,13,0,No change
279,S-2024-2025,2,10,T-004-T,20,3,3,14,16,41,-25,12,0,No change


In [96]:
# cols_matches = ['date', 'attendance', 'season_id']

# df_matches = df_m.set_index('match_id')[cols_matches]
df_matches = df_m.copy()
df_matches


,season_id,season,tier,division,match_id,match_name,date,attendance,home_team_id,home_team_name,...,score,home_team_score,away_team_score,home_team_score_margin,away_team_score_margin,home_team_win,away_team_win,draw,result,note
0,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,2011-04-13,"2,510",T-008-T,Chelsea Ladies,...,0 -- 1,0,1,-1,1,0,1,0,Away team win,NaN
1,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,2011-04-13,742,T-016-T,Lincoln Ladies,...,0 -- 1,0,1,-1,1,0,1,0,Away team win,NaN
2,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-003-M,Birmingham City Ladies vs Bristol Academy,2011-04-14,602,T-003-T,Birmingham City Ladies,...,4 -- 0,4,0,4,-4,1,0,0,Home team win,NaN
3,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-004-M,Liverpool Ladies vs Everton Ladies,2011-04-14,835,T-017-T,Liverpool Ladies,...,3 -- 3,3,3,0,0,0,0,1,Draw,NaN
4,S-2011-2011-1-S,2011-2011,1,FA Women's Super League (WSL),M-2011-2011-1-005-M,Everton Ladies vs Doncaster Rovers Belles,2011-04-20,220,T-013-T,Everton Ladies,...,1 -- 1,1,1,0,0,0,0,1,Draw,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2535,S-2024-2025-2-S,2024-2025,2,Women's Championship,M-2024-2025-2-106-M,Birmingham City Women vs London City Lionesses,2025-05-04,8749,T-003-T,Birmingham City Women,...,2 -- 2,2,2,0,0,0,0,1,Draw,NaN
2536,S-2024-2025-2-S,2024-2025,2,Women's Championship,M-2024-2025-2-107-M,Blackburn Rovers Women vs Newcastle United Women,2025-05-04,723,T-004-T,Blackburn Rovers Women,...,1 -- 5,1,5,-4,4,0,1,0,Away team win,NaN
2537,S-2024-2025-2-S,2024-2025,2,Women's Championship,M-2024-2025-2-108-M,Durham Women vs Bristol City Women,2025-05-04,1031,T-012-T,Durham Women,...,2 -- 0,2,0,2,-2,1,0,0,Home team win,NaN
2538,S-2024-2025-2-S,2024-2025,2,Women's Championship,M-2024-2025-2-109-M,Southampton Women vs Charlton Athletic Women,2025-05-04,2188,T-026-T,Southampton Women,...,0 -- 2,0,2,-2,2,0,1,0,Away team win,NaN


In [ ]:
drop_cols_appearences = ['match_name', 'date', 'attendance']

df_appearances = df_a.drop(columns=cols_seasons+cols_matches)
df_appearances


,match_id,match_name,team_id,team_name,opponent_id,opponent_name,home_team,away_team,goals_for,goals_against,goal_difference,result,win,loss,draw,note,points
0,M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,T-008-T,Chelsea Ladies,T-001-T,Arsenal Ladies,1,0,0,1,-1,Loss,0,1,0,NaN,0
1,M-2011-2011-1-001-M,Chelsea Ladies vs Arsenal Ladies,T-001-T,Arsenal Ladies,T-008-T,Chelsea Ladies,0,1,1,0,1,Win,1,0,0,NaN,3
2,M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,T-016-T,Lincoln Ladies,T-011-T,Doncaster Rovers Belles,1,0,0,1,-1,Loss,0,1,0,NaN,0
3,M-2011-2011-1-002-M,Lincoln Ladies vs Doncaster Rovers Belles,T-011-T,Doncaster Rovers Belles,T-016-T,Lincoln Ladies,0,1,1,0,1,Win,1,0,0,NaN,3
4,M-2011-2011-1-003-M,Birmingham City Ladies vs Bristol Academy,T-003-T,Birmingham City Ladies,T-006-T,Bristol Academy,1,0,4,0,4,Win,1,0,0,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5075,M-2024-2025-2-108-M,Durham Women vs Bristol City Women,T-006-T,Bristol City Women,T-012-T,Durham Women,0,1,0,2,-2,Loss,0,1,0,NaN,0
5076,M-2024-2025-2-109-M,Southampton Women vs Charlton Athletic Women,T-026-T,Southampton Women,T-007-T,Charlton Athletic Women,1,0,0,2,-2,Loss,0,1,0,NaN,0
5077,M-2024-2025-2-109-M,Southampton Women vs Charlton Athletic Women,T-007-T,Charlton Athletic Women,T-026-T,Southampton Women,0,1,2,0,2,Win,1,0,0,NaN,3
5078,M-2024-2025-2-110-M,Sunderland Women vs Sheffield United Women,T-027-T,Sunderland Women,T-024-T,Sheffield United Women,1,0,2,1,1,Win,1,0,0,NaN,3
